In [1]:
import csv
import sklearn.feature_extraction.text as text;
import numpy as np
import scipy.sparse as sparse
from sklearn import linear_model
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import cosine_similarity
import threading  
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

    
class Loader:
    def __init__(self,):
        train_file = 'train.csv';
        test_file = 'test.csv';
        self.item2id = {}
        self.id2item = {}
        self.item_title = {}
        self.item_text = {}
        self.test_id = []
        self.train = self.load(train_file);
        self.test = self.load(test_file);
        self.load_description();
        self.title = []
        
        print('begin proprocessing')
        self.preprocess();
        print('compute cosine')
        self.train_cosine = self.compute_cosine([self.train_s, self.item_title_tfidf, self.item_text_tfidf], self.train);
        self.train_cosine = np.asarray(self.train_cosine);
        self.test_cosine = self.compute_cosine([self.test_s, self.item_title_tfidf, self.item_text_tfidf], self.test);
        self.test_cosine = np.asarray(self.test_cosine)
        print('finish')
        
        
    def load(self, file_name):
        reader = csv.reader(open(file_name, encoding='latin-1'));
        cnt = 0;
        data = []
        for row in reader:
            cnt += 1;
            if (cnt == 1):
                continue;
            if(file_name == "test.csv"): self.test_id.append(row[0])
            #self.item
            item = int(row[1]);
            if (item not in self.item2id):
                idx = len(self.item2id)
                self.item2id[item] = idx;
                self.id2item[idx] = item;
                self.item_title[idx] = row[2].lower();    
            sample = {'id': int(row[0]), 'x': self.item2id[item], 'y': row[3].lower()}
            if (len(row) == 5):
                rate = float(row[-1])
                sample['r'] = rate;
            data.append(sample);
        return data;
    
    def load_description(self,):
        reader = csv.reader(open('product_descriptions.csv', encoding='latin-1'));
        cnt = 0;
        for row in reader:
            cnt += 1;
            if (cnt == 1):
                continue;
            
            item = int(row[0]);
            idx = self.item2id[item]
            self.item_text[idx] = row[1].lower()
        return ;


    def preprocess(self,):
        transformer = text.TfidfVectorizer(max_features = 270000, stop_words ='english',tokenizer=LemmaTokenizer());
        n = len(self.item2id);
        corpus = []
        item_title = []
        item_text = []
        for i in range(n):
            corpus.append(self.item_text[i] + ' ' + self.item_title[i]);
            item_title.append(self.item_title[i])
            item_text.append(self.item_text[i])
            #self.title.append(wordcount(self.item_title[i],self.item_text[i]))
        self.title = np.asarray(self.title)
        transformer.fit(corpus);
        self.item_title_tfidf = transformer.transform(item_title);
        self.item_text_tfidf = transformer.transform(item_text);
        train_s = []
        for item in self.train:
            train_s.append(item['y']);
        test_s = []
        for item in self.test:
            test_s.append(item['y']);
        self.train_s = transformer.transform(train_s);
        self.test_s = transformer.transform(test_s);
        
    def compute_cosine(self, inputs, data):
        cosine = []
        print(inputs[0].shape[0])
        for i in range(inputs[0].shape[0]):
            x = data[i]['x'];
            a = inputs[0][i];b = inputs[1][x];c = inputs[2][x];
            ab = cosine_similarity(a,b)[0][0];
            bc = cosine_similarity(b,c)[0][0];
            ac = cosine_similarity(a,c)[0][0];
            cosine.append([ab,bc,ac]);
        return cosine

In [2]:
Data = Loader()

begin proprocessing
compute cosine
74067
166693
finish


In [15]:
def wordcount(a,b):
    count = 0
    for ch in a:
        if ch in b:
            count += b.count(ch)
    return count
#self.item_title[item['x']]
#print(Data.title.shape,Data.title[:10])

In [51]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
def fuzzy(s1, s2):
    return fuzz.token_set_ratio(s1, s2) / 100.0

train_r = [];
train_t = []
len_train_y = []
#title:query  number of common words
common = []
#description:query  number of common words
common1 = []
fuzz_title = []
fuzz_des = []
for i in range(len(Data.train)):
    item = Data.train[i];
    train_r.append(item['r']);
    train_t.append(item['x'])
    len_train_y.append(len(item['y'].split()))
    common.append(wordcount(Data.item_title[item['x']].split(),item['y'].split()))
    common1.append(wordcount(Data.item_text[item['x']].split(),item['y'].split()))
    fuzz_title.append(fuzzy(Data.item_title[item['x']].split(),item['y'].split()))
    fuzz_des.append(fuzzy(Data.item_title[item['x']].split(),item['y'].split()))
    
    #wordcount(Data.item_title[item['id']],item['y'])
#train_t = Data.item_tfidf[train_t]
#train_s = sparse.hstack([train_t, Data.train_s])
print(common[:100])


#########
r,c = Data.train_cosine.shape
train_s = np.zeros((r,5))
#train_s[:,:-5] = Data.train_cosine
train_s[:,-5] = fuzz_des
train_s[:,-4] = fuzz_title
train_s[:,-3] = common1
train_s[:,-2] = common
#train_s = Data.train_cosine

for i in range(len(train_s)):
    train_s[i][-1] = len_train_y[i]
    
    
    
#train_s = Data.train_cosine.reshape(-1,4)
#b.reshape(-1,3)

# print(b.shape)
print(train_s.shape)
# print(b[:3])
# print(train_s[:3])
train_r = np.asarray(train_r)
#avg_r = np.mean(train_r);
#train_r = train_r - avg_r;

[1, 0, 0, 1, 3, 1, 2, 0, 1, 2, 0, 4, 1, 1, 3, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 0, 0, 0, 4, 2, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 3, 0, 1, 0, 0, 0, 1, 0, 0, 2, 4, 2, 1, 1, 1, 2, 1, 1, 1, 2, 4, 4, 1, 1, 0, 2, 0, 1, 3, 2, 2, 2, 1, 1, 3, 0, 0, 2, 3, 2, 3, 2, 2, 2, 3, 0, 3, 1, 2, 1, 1, 0]
(74067, 5)


In [23]:


# train_r = [];
# train_t = []
# len_train_y = []
# #title:query  number of common words
# common = []
# #description:query  number of common words
# common1 = []

# for i in range(len(Data.train)):
#     item = Data.train[i];
#     train_r.append(item['r']);
#     train_t.append(item['x'])
#     len_train_y.append(len(item['y'].split()))
#     common.append(wordcount(Data.item_title[item['x']].split(),item['y'].split()))
#     common1.append(wordcount(Data.item_text[item['x']].split(),item['y'].split()))
    
#     #wordcount(Data.item_title[item['id']],item['y'])
# #train_t = Data.item_tfidf[train_t]
# #train_s = sparse.hstack([train_t, Data.train_s])
# print(common[:100])


# #########
# r,c = Data.train_cosine.shape
# #train_s = np.zeros((r,c+5))
# #train_s = np.zeros((r,c+5))
# train_s = np.zeros((r,5))
# train_s[:,:-5] = Data.train_cosine
# train_s[:,-5] = fuzz_title_test
# train_s[:,-4] = fuzz_des_test
# train_s[:,-3] = common1
# train_s[:,-2] = common
# #train_s = Data.train_cosine

# for i in range(len(train_s)):
#     train_s[i][-1] = len_train_y[i]
    
    
    
# #train_s = Data.train_cosine.reshape(-1,4)
# #b.reshape(-1,3)

# # print(b.shape)
# print(train_s.shape)
# # print(b[:3])
# # print(train_s[:3])
# train_r = np.asarray(train_r)
# #avg_r = np.mean(train_r);
# #train_r = train_r - avg_r;

[1, 0, 0, 1, 3, 1, 2, 0, 1, 2, 0, 4, 1, 1, 3, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 0, 0, 0, 4, 2, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 3, 0, 1, 0, 0, 0, 1, 0, 0, 2, 4, 2, 1, 1, 1, 2, 1, 1, 1, 2, 4, 4, 1, 1, 0, 2, 0, 1, 3, 2, 2, 2, 1, 1, 3, 0, 0, 2, 3, 2, 3, 2, 2, 2, 3, 0, 3, 1, 2, 1, 1, 0]
(74067, 6)


In [43]:
#lr section

model = linear_model.Ridge(alpha = 30.)
print(train_s.shape)
model.fit(train_s, train_r)
print(model.get_params(deep=True))
print(model.coef_ )

(74067, 8)
{'alpha': 30.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}
[ 0.42631762 -0.26884102  0.46593481  0.16730788  0.16730788  0.00332264
 -0.00132144 -0.04340472]


In [44]:
print(model.score(train_s,train_r))

0.1379982062806514


In [50]:

test_t = []
len_test_y = []
common_test = []
common_test1 = []
fuzz_title_test = []
fuzz_des_test = []
for i in range(len(Data.test)):
    if(i%1000==0):print(i)
    item = Data.test[i];
    test_t.append(item['x'])
    len_test_y.append(len(item['y'].split()))
    common_test.append(wordcount(Data.item_title[item['x']].split(),item['y'].split()))
    common_test1.append(wordcount(Data.item_text[item['x']].split(),item['y'].split()))
    fuzz_title_test.append(fuzzy(Data.item_text[item['x']].split(),item['y'].split()))
    fuzz_des_test.append(fuzzy(Data.item_text[item['x']].split(),item['y'].split()))
    
#test_t = Data.item_tfidf[test_t]
#test_t = Data.item_tfidf[test_t]
#test_s = sparse.hstack([test_t, Data.test_s])

#
r,c = Data.test_cosine.shape
print(r,c)
#test_s = np.zeros((r,c+5))
#test_s[:,:-5] = Data.test_cosine
test_s = np.zeros((r,5))
test_s[:,-5] = fuzz_des_test
test_s[:,-4] = fuzz_title_test
test_s[:,-3] = common_test1
test_s[:,-2] = common_test

print(test_s.shape)


for i in range(len(len_test_y)):
    test_s[i][-1] = len_test_y[i]


#test_s = Data.test_cosine.reshape(-1,3)
test_r = model.predict(test_s)

print(test_s.shape)
print(test_r)
len(test_r)
for i in range(len(test_r)):
    if (test_r[i]<1): test_r[i] = 1
    if (test_r[i]>3): test_r[i] = 3
op = np.asarray(test_r)
np.savetxt("submission.csv", op, delimiter=",")
f = open("submission.csv", "w")
f.write("{},{}\n".format("id","relevance"))

# true_ids = []
# for i in ids:
#     true_ids.append(Data.id2item[i])
for i in range(len(test_r)):
    f.write("{},{}\n".format(Data.test_id[i], test_r[i]))

f.close()



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
print("end of lr")

In [ ]:
#fuzzy

In [11]:
#random forest setion
from sklearn.ensemble import RandomForestRegressor
model2 =RandomForestRegressor(max_depth=2, random_state=0)
model2.fit(train_s, train_r)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:

print("finished")
test_t = []
len_test_y = []
common_test = []
for i in range(len(Data.test)):
    item = Data.test[i];
    test_t.append(item['x'])
    len_test_y.append(len(item['y'].split()))
    common_test.append(wordcount(Data.item_title[item['x']].split(),item['y'].split()))
#test_t = Data.item_tfidf[test_t]
#test_s = sparse.hstack([test_t, Data.test_s])

#
r,c = Data.test_cosine.shape
print(r,c)
test_s = np.zeros((r,c+2))
test_s[:,-2] = common_test
test_s[:,:-2] = Data.test_cosine


for i in range(len(len_test_y)):
    test_s[i][-1] = len_test_y[i]


#test_s = Data.test_cosine.reshape(-1,3)
test_r = model2.predict(test_s)

print(test_s.shape)
print(test_r)
len(test_r)
for i in range(len(test_r)):
    if (test_r[i]<1): test_r[i] = 1
    if (test_r[i]>3): test_r[i] = 3
op = np.asarray(test_r)
np.savetxt("submission.csv", op, delimiter=",")
f = open("submission.csv", "w")
f.write("{},{}\n".format("id","relevance"))

# true_ids = []
# for i in ids:
#     true_ids.append(Data.id2item[i])
for i in range(len(test_r)):
    f.write("{},{}\n".format(Data.test_id[i], test_r[i]))

f.close()


finished
166693 3
(166693, 5)
[2.17164586 2.10572163 2.29484725 ... 2.61870331 2.47921693 2.59006313]
